In [1]:
# COBA 1

import os
import numpy as np
import pandas as pd
from scipy.io import wavfile
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Fungsi untuk ekstraksi fitur menggunakan metode coarse grain
def coarse_grain(signal, num_windows):
    window_size = len(signal) // num_windows
    coarse_grained_signal = []
    for i in range(num_windows):
        start = i * window_size
        end = start + window_size
        window = signal[start:end]
        coarse_grained_signal.append(np.mean(window))
    return coarse_grained_signal

# Fungsi untuk ekstraksi fitur menggunakan metode Hjorth
def hjorth(signal):
    first_diff = np.diff(signal.ravel())
    second_diff = np.diff(signal.ravel(), 2)
    activity = np.var(signal)
    mobility = np.sqrt(np.var(first_diff) / activity) if activity > 0 else 0.0
    complexity = np.sqrt(np.var(second_diff) / np.var(first_diff)) if np.var(first_diff) > 0 else 0.0
    return activity, mobility, complexity

# Mendapatkan daftar file suara detak jantung normal dan abnormal
# normal_folder = 'training/normal'
# abnormal_folder = 'training/abnormal'


normal_folder = 'dataset/training-a/wav copy/normal'
abnormal_folder = 'dataset/training-a/wav copy/abnormal'

normal_files = [os.path.join(normal_folder, file) for file in os.listdir(normal_folder)]
abnormal_files = [os.path.join(abnormal_folder, file) for file in os.listdir(abnormal_folder)]

# Daftar fitur dan label
features = []
labels = []

# Memproses file suara detak jantung normal
for file_path in normal_files:
    rate, data = wavfile.read(file_path)
    
    # Ekstraksi fitur menggunakan metode coarse grain
    coarse_grained = coarse_grain(data, 10)
    file_features = []
    for window in coarse_grained:
        window_features = hjorth(window)
        file_features.append(window_features)
    
    # Menambahkan fitur dan label ke daftar
    features.extend(file_features)
    labels.extend(['normal'] * len(file_features))

# Memproses file suara detak jantung abnormal
for file_path in abnormal_files:
    rate, data = wavfile.read(file_path)
    
    # Ekstraksi fitur menggunakan metode coarse grain
    coarse_grained = coarse_grain(data, 20)
    file_features = []
    for window in coarse_grained:
        window_features = hjorth(window)
        file_features.append(window_features)
    
    # Menambahkan fitur dan label ke daftar
    features.extend(file_features)
    labels.extend(['abnormal'] * len(file_features))

# Konversi label menjadi bilangan numerik
label_encoder = LabelEncoder()
# encoded_labels = label_encoder.fit_transform(labels)

encoded_labels = np_utils.to_categorical(label_encoder.fit_transform(labels))

# Ubah menjadi array numpy
features = np.array(features)
encoded_label= np.array(encoded_labels)

# Reshape data sesuai dengan input LSTM
features = np.reshape(features, (features.shape[0], features.shape[1], 1))



# Split data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.2, random_state=42)



# Buat model LSTM

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint,TensorBoard,ProgbarLogger
from keras.utils import np_utils
from sklearn import metrics 
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import itertools

import tensorflow.keras.backend as K

def cosine_similarity(y_true, y_pred):
    y_true = K.l2_normalize(y_true, axis=-1)
    y_pred = K.l2_normalize(y_pred, axis=-1)
    return K.mean(y_true * y_pred, axis=-1)


CLASSES = ['normal','abnormal']

print('Build LSTM RNN model ...')
model = Sequential()
model.add(LSTM(units=64, dropout=0.05, recurrent_dropout=0.20, return_sequences=True,input_shape = (3,1)))
model.add(LSTM(units=32, dropout=0.05, recurrent_dropout=0.20, return_sequences=False))
model.add(Dense(len(CLASSES), activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='Adamax', metrics=['acc','mse', 'mae', 'mape', 'cosine'])

model.compile(loss='categorical_crossentropy', optimizer='Adamax', metrics=['acc','mse', 'mae', 'mape', cosine_similarity])
model.summary()

# %%time
# saved model checkpoint file
best_model_file="best_model_trained.hdf5"
#train_model_file=file_path+"/checkpoints/weights.best_{epoch:02d}-{loss:.2f}.hdf5"
MAX_PATIENT=12
MAX_EPOCHS=100
MAX_BATCH=32

# callbacks
# removed EarlyStopping(patience=MAX_PATIENT)
callback=[ReduceLROnPlateau(patience=MAX_PATIENT, verbose=1),
          ModelCheckpoint(filepath=best_model_file, monitor='acc', verbose=1, save_best_only=True)]

print ("training started..... please wait.")
# training
history=model.fit(X_train, y_train, 
                  batch_size=MAX_BATCH, 
                  epochs=MAX_EPOCHS,
                  verbose=0,
                  validation_data=(X_test, y_test),
                  callbacks=callback) 

print ("training finised!")


# %%time
#Plot Keras History
#Plot loss and accuracy for the training and validation set.
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    plt.figure(figsize=(22,10))
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    ## Accuracy
    plt.figure(221, figsize=(20,10))
    ## Accuracy
    # plt.figure(2,figsize=(14,5))
    plt.subplot(221, title='Accuracy')
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Loss (' + str(format(history.history[l][-1],'.5f'))+')')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    ## Loss
    plt.subplot(222, title='Loss')
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

# plot history
plot_history(history)

c:\Users\dianr\.conda\envs\daily\lib\site-packages\numpy\core\fromnumeric.py:3715: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\dianr\.conda\envs\daily\lib\site-packages\numpy\core\_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
c:\Users\dianr\.conda\envs\daily\lib\site-packages\numpy\core\_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


NameError: name 'np_utils' is not defined

In [21]:
import numpy as np
import scipy.signal as signal
from scipy.stats import skew as skewness
from scipy.stats import kurtosis as kurtosis_value
import pandas as pd

import librosa
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.layers import LSTM, Dense, Dropout
from keras.regularizers import l2


dataset_audio = pd.read_csv("dataset heartbeat.csv") 
audio_paths = []
labels = []

for i in range(len(dataset_audio)):
    file_name = "dataset/training-a/wav/" + dataset_audio['Data'][i] + ".wav"
    audio_paths.append(file_name)

    print(file_name)

    # labels.append(dataset_audio['Column2'][i])
    labels.append(dataset_audio['Label'][i])



# Pra-pemrosesan audio dan pemotongan segmen
def preprocess_audio(audio_path, segment_length, overlap):
    audio, sr = librosa.load(audio_path, sr=None)
    audio_normalized = audio / np.max(np.abs(audio))  # Normalisasi amplitudo audio

    segments = []
    step = int(segment_length * (1 - overlap))
    for i in range(0, len(audio_normalized) - segment_length, step):
        segment = audio_normalized[i:i + segment_length]
        segments.append(segment)
    return segments


# Fungsi untuk menghitung fitur-fitur Hjorth
def hjorth_features(signal):
    diff1 = np.diff(signal)
    diff2 = np.diff(signal, 2)
    var_signal = np.var(signal)
    var_diff1 = np.var(diff1)
    var_diff2 = np.var(diff2)
    mobility = np.sqrt(var_diff1 / var_signal)
    complexity = np.sqrt(var_diff2 / var_diff1) / mobility if var_diff1 != 0 else 0
    return np.mean(signal), np.std(signal), mobility, complexity, np.mean(diff1), np.std(diff1), np.mean(diff2), np.std(diff2)


# Pra-pemrosesan audio dan ekstraksi fitur Hjorth
segment_length = 1000  # Panjang segmen suara (misalnya 1 detik)
overlap = 0.5  # Jumlah tumpang tindih antara segmen-suara
features = []


for audio_path in audio_paths:
    audio, sr = librosa.load(audio_path, sr=None)
    audio_normalized = audio / np.max(np.abs(audio))  # Normalisasi amplitudo audio
    mean, std, mobility, complexity, mean_diff1, std_diff1, mean_diff2, std_diff2 = hjorth_features(audio_normalized)
    features.append([mean, std, mobility, complexity, mean_diff1, std_diff1, mean_diff2, std_diff2])




dataset/training-a/wav/a0001.wav
dataset/training-a/wav/a0002.wav
dataset/training-a/wav/a0003.wav
dataset/training-a/wav/a0004.wav
dataset/training-a/wav/a0005.wav
dataset/training-a/wav/a0006.wav
dataset/training-a/wav/a0007.wav
dataset/training-a/wav/a0008.wav
dataset/training-a/wav/a0009.wav
dataset/training-a/wav/a0010.wav
dataset/training-a/wav/a0011.wav
dataset/training-a/wav/a0012.wav
dataset/training-a/wav/a0013.wav
dataset/training-a/wav/a0014.wav
dataset/training-a/wav/a0015.wav
dataset/training-a/wav/a0016.wav
dataset/training-a/wav/a0017.wav
dataset/training-a/wav/a0018.wav
dataset/training-a/wav/a0019.wav
dataset/training-a/wav/a0020.wav
dataset/training-a/wav/a0021.wav
dataset/training-a/wav/a0022.wav
dataset/training-a/wav/a0023.wav
dataset/training-a/wav/a0024.wav
dataset/training-a/wav/a0025.wav
dataset/training-a/wav/a0026.wav
dataset/training-a/wav/a0027.wav
dataset/training-a/wav/a0028.wav
dataset/training-a/wav/a0029.wav
dataset/training-a/wav/a0030.wav
dataset/tr

In [30]:
features = np.array(features)
labels = np.array(labels)

# Melakukan label encoding pada label kelas
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)



# Split dataset menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(features, labels_encoded, test_size=0.2, random_state=42)

# Mengubah bentuk data menjadi 3 dimensi untuk RNN-LSTM (jumlah sampel, panjang setiap sampel, jumlah fitur)
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))


In [52]:
# % pylab inline
import os
import pandas as pd
import librosa
import librosa.display as display
import glob 

def extract_data(folder):
    # function to load files and extract features
    file_names = glob.glob(os.path.join(folder, '*.wav'))
    data = []
    print(file_names)
    for file_name in file_names:
        try:
            # here kaiser_fast is a technique used for faster extraction
            X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0) 
        except Exception as e:
            print("Error encountered while parsing file: ", file_name)
        
        feature = np.array(mfccs).reshape([-1,1])
        data.append(feature)
    return data


normal_onehot = 1
murmur_onehot = 0

normal_sounds = extract_data("dataset/training-ass/wav copy/normal")
normal_labels = [normal_onehot for items in normal_sounds]
murmur_sounds = extract_data("dataset/training-ass/wav copy/abnormal")
# x_test = extract_data("test")
murmur_labels = [murmur_onehot for items in murmur_sounds]

train_sounds = np.concatenate((normal_sounds, murmur_sounds))
train_labels = np.concatenate((normal_labels, murmur_labels))

# x_train, x_test, y_train, y_test = train_test_split(np.concatenate((normal_sounds, murmur_sounds)), np.concatenate((normal_labels, murmur_labels)), test_size=0.2, random_state=42)
# x_train, x_test, y_train, y_test = train_test_split(features, labels_encoded, test_size=0.2, random_state=42)


# # x_train = np.concatenate((normal_sounds, murmur_sounds))
# # y_train = np.concatenate((normal_labels, murmur_labels))
# x_train[5].shape

['dataset/training-ass/wav copy/normal\\normal_a0007.wav', 'dataset/training-ass/wav copy/normal\\normal_a0009.wav', 'dataset/training-ass/wav copy/normal\\normal_a0011.wav', 'dataset/training-ass/wav copy/normal\\normal_a0012.wav', 'dataset/training-ass/wav copy/normal\\normal_a0016.wav', 'dataset/training-ass/wav copy/normal\\normal_a0019.wav', 'dataset/training-ass/wav copy/normal\\normal_a0025.wav', 'dataset/training-ass/wav copy/normal\\normal_a0027.wav', 'dataset/training-ass/wav copy/normal\\normal_a0028.wav', 'dataset/training-ass/wav copy/normal\\normal_a0029.wav', 'dataset/training-ass/wav copy/normal\\normal_a0032.wav', 'dataset/training-ass/wav copy/normal\\normal_a0035.wav', 'dataset/training-ass/wav copy/normal\\normal_a0038.wav', 'dataset/training-ass/wav copy/normal\\normal_a0049.wav', 'dataset/training-ass/wav copy/normal\\normal_a0050.wav', 'dataset/training-ass/wav copy/normal\\normal_a0053.wav', 'dataset/training-ass/wav copy/normal\\normal_a0055.wav', 'dataset/trai

(8,)

In [55]:
x_train, x_test, y_train, y_test = train_test_split(features, labels_encoded, test_size=0.2, random_state=42)


# x_train = np.concatenate((normal_sounds, murmur_sounds))
# y_train = np.concatenate((normal_labels, murmur_labels))
x_train[5].shape

# Mengubah bentuk data menjadi 3 dimensi untuk RNN-LSTM (jumlah sampel, panjang setiap sampel, jumlah fitur)
x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))

x_train[5].shape

(1, 8)

In [60]:
print('Build LSTM RNN model ...')
model = Sequential()
model.add(LSTM(units=64, dropout=0.05, recurrent_dropout=0.35, return_sequences=True,input_shape = (1, 8)))
model.add(LSTM(units=32, dropout=0.05, recurrent_dropout=0.35, return_sequences=False))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=32, epochs=100)

model.summary()


Build LSTM RNN model ...
Train on 327 samples
Epoch 1/100
327/327 [==============================] - 1s 3ms/sample - loss: 0.6701 - acc: 0.7125
Epoch 2/100
327/327 [==============================] - 0s 326us/sample - loss: 0.6345 - acc: 0.7125
Epoch 3/100
327/327 [==============================] - 0s 273us/sample - loss: 0.5989 - acc: 0.7125
Epoch 4/100
327/327 [==============================] - 0s 220us/sample - loss: 0.5885 - acc: 0.7125
Epoch 5/100
327/327 [==============================] - 0s 205us/sample - loss: 0.5850 - acc: 0.7125
Epoch 6/100
327/327 [==============================] - 0s 190us/sample - loss: 0.5812 - acc: 0.7125
Epoch 7/100
327/327 [==============================] - 0s 181us/sample - loss: 0.5831 - acc: 0.7125
Epoch 8/100
327/327 [==============================] - 0s 183us/sample - loss: 0.5862 - acc: 0.7125
Epoch 9/100
327/327 [==============================] - 0s 188us/sample - loss: 0.5840 - acc: 0.7125
Epoch 10/100
327/327 [==============================] - 

In [61]:
# Melakukan prediksi pada data uji
predictions = model.predict(x_test)
y_pred = np.argmax(predictions, axis=1)

# Mengembalikan label kelas dari label encoding
y_test = label_encoder.inverse_transform(y_test)
y_pred = label_encoder.inverse_transform(y_pred)

# Menghitung akurasi
accuracy = accuracy_score(y_test, y_pred)
print("Akurasi:", accuracy)

c:\Users\dianr\.conda\envs\daily\lib\site-packages\keras\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
c:\Users\dianr\.conda\envs\daily\lib\site-packages\numpy\lib\arraysetops.py:608: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)


ValueError: y contains previously unseen labels: ['abnormal' 'normal']

In [4]:
dataset_audio = pd.read_csv("dataset heartbeat.csv") 
dataset_audio

,Data,Column2,Label
0,a0001,1,abnormal
1,a0002,1,abnormal
2,a0003,1,abnormal
3,a0004,1,abnormal
4,a0005,1,abnormal
...,...,...,...
404,a0405,-1,normal
405,a0406,-1,normal
406,a0407,-1,normal
407,a0408,-1,normal


## YANG LAIN

In [ ]:
# Importing required libraries
import numpy as np
import pandas as pd
import librosa
import os
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# for data transformation
import numpy as np
import pandas as pd
# for visualizing the data
import matplotlib.pyplot as plt
# for opening the media file
import scipy.io.wavfile as wavfile
import wave
from sklearn.model_selection import train_test_split
# from pyAudioAnalysis import audioTrainTest as aT

from IPython.display import Audio as ipd

import librosa
import librosa.display

from sklearn.preprocessing import StandardScaler
# gather software versions
from scipy.signal import resample,butter, filtfilt
import keras


In [2]:
def read_audio_files(path, sample_rate=44100):
    audio_files = []
    for filename in os.listdir(path):
        if filename.endswith('.wav'):
            file_path = os.path.join(path, filename)
            sample_rate, data = wavfile.read(file_path)
            audio_files.append(data)
    return audio_files

In [3]:
def coarse_grain(data, level):
    data = np.array(data)
    coeffs = wavedec(data, 'haar', level=level)
    coeffs[-1] = np.zeros_like(coeffs[-1])
    coeffs[-2] = np.zeros_like(coeffs[-2])
    coeffs[-3] = np.zeros_like(coeffs[-3])
    coeffs[0] = np.zeros_like(coeffs[0])
    return np.concatenate(coeffs)

In [4]:
def hjorth_desc(data):
    first_deriv = np.diff(data)
    second_deriv = np.diff(data, 2)
    activity = np.var(data)
    mobility = np.sqrt(np.var(first_deriv) / activity)
    complexity = np.sqrt(np.var(second_deriv) / np.var(first_deriv))
    return [activity, mobility, complexity]

In [17]:
def extract_features(X, coarse_grain_level):
    # Coarse Grain
    cg = []
    for x in X:
        x_cg = np.abs(x[:coarse_grain_level])  # downsample by the specified factor
        cg.append(x_cg)
    cg = np.array(cg)
    # Hjorth Descriptor
    hj = []
    for x in X:
        x_hj = pd.Series(x).apply(lambda x: [x, x**2]).apply(pd.Series).agg(['sum', 'mean', 'std']).values
        hj.append(x_hj)
    hj = np.array(hj)
    # Concatenate features
    features = np.concatenate([cg, hj], axis=1)
    return features

In [18]:
# Defining function to create dataset
def create_dataset(path):
    features = []
    labels = []
    for label in os.listdir(path): # Loop over all labels
        label_path = os.path.join(path, label)
        for file in os.listdir(label_path): # Loop over all files in label directory
            file_path = os.path.join(label_path, file)
            feature = extract_features(file_path) # Extract features
            features.append(feature)
            labels.append(label) # Add label to list
    return np.array(features), np.array(labels)

In [19]:
normal_path = 'dataset/training-a/wav copy/normal'
abnormal_path = 'dataset/training-a/wav copy/abnormal'
normal_files = read_audio_files(normal_path)
abnormal_files = read_audio_files(abnormal_path)
normal_labels = np.zeros(len(normal_files))
abnormal_labels = np.ones(len(abnormal_files))
all_files = np.concatenate((normal_files, abnormal_files))
all_labels = np.concatenate((normal_labels, abnormal_labels))


In [20]:
# # features, labels = create_dataset(path)
# features_scaled = scaler.fit_transform(features)
# X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels, test_size=0.2, random_state=42)

In [21]:
features, labels = extract_features(all_files, all_labels, coarse_grain_level=3)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)
X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels, test_size=0.3, random_state=42)


TypeError: extract_features() got multiple values for argument 'coarse_grain_level'

In [ ]:
clf = SVC(kernel='rbf', C=10, gamma=0.1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.67      0.09      0.15        46
         1.0       0.64      0.97      0.77        77

    accuracy                           0.64       123
   macro avg       0.65      0.53      0.46       123
weighted avg       0.65      0.64      0.54       123



In [ ]:
mnb = multino
mnb.fit(X_train, y_train)
y_pred = mnb.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.67      0.09      0.15        46
         1.0       0.64      0.97      0.77        77

    accuracy                           0.64       123
   macro avg       0.65      0.53      0.46       123
weighted avg       0.65      0.64      0.54       123



In [ ]:
path = 'dataset/training-a/wav copy/'
features, labels = create_dataset(path)

TypeError: extract_features() missing 2 required positional arguments: 'labels' and 'coarse_grain_level'

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [ ]:
# Creating LSTM model
model = Sequential()
model.add(LSTM(units=64, input_shape=X_train.shape[1:], return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=32))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid'))

# Compiling model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Predicting labels for test set
y_pred = model.predict(X_test)
y_pred = [1 if y>=0.5 else 0 for y in y_pred]

# Calculating accuracy score
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

ValueError: Input 0 of layer "lstm_2" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 22055)

In [ ]:
import numpy as np
import pandas as pd
import librosa
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping



In [ ]:

# Load audio data and labels
def load_data():
    # normal_path = 'path/to/normal/audio/files'
    # abnormal_path = 'path/to/abnormal/audio/files'
    normal_path = 'dataset/training-a/wav copy/normal'
    abnormal_path = 'dataset/training-a/wav copy/abnormal'
    normal_files = librosa.util.find_files(normal_path)
    abnormal_files = librosa.util.find_files(abnormal_path)
    normal_data = [librosa.load(f, sr=None)[0] for f in normal_files]
    abnormal_data = [librosa.load(f, sr=None)[0] for f in abnormal_files]
    X = normal_data + abnormal_data
    y = np.concatenate([np.zeros(len(normal_data)), np.ones(len(abnormal_data))])
    return X, y



In [ ]:
# Extract features using Coarse Grain and Hjorth Descriptor
def extract_features(X):
    # Coarse Grain
    cg = []
    for x in X:
        x_cg = np.abs(x[::100])  # downsample by a factor of 100
        cg.append(x_cg)
    cg = np.array(cg)
    # Hjorth Descriptor
    hj = []
    for x in X:
        x_hj = pd.Series(x).apply(lambda x: [x, x**2]).apply(pd.Series).agg(['sum', 'mean', 'std']).values
        hj.append(x_hj)
    hj = np.array(hj)
    # Concatenate features
    features = np.concatenate([cg, hj], axis=1)
    return features



In [ ]:
# Split data into train and test sets
X, y = load_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Extract features from audio data
X_train = extract_features(X_train)
X_test = extract_features(X_test)


C:\Users\dianr\AppData\Local\Temp\ipykernel_18516\1807999537.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cg = np.array(cg)


AxisError: axis 1 is out of bounds for array of dimension 1

In [ ]:
# Reshape input data to be 3D for LSTM
timesteps = 1
features = X_train.shape[1]
X_train = X_train.reshape(X_train.shape[0], timesteps, features)
X_test = X_test.reshape(X_test.shape[0], timesteps, features)

In [ ]:
# Build LSTM model
model = Sequential()
model.add(LSTM(units=64, input_shape=(timesteps, features), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=32, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid'))

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='min')

# Train model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Evaluate model on test data
loss, accuracy = model.evaluate(X_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)